# Collect all Paths 
This does not need to be ran anymore we have the list in paths.txt now. Just read that into simulation_paths and you are good to go with collecting data

In [210]:
import s3fs
import zarr
import json
from tqdm import tqdm
import pandas as pd
import pickle
from pprint import pprint
import os

In [265]:
endpoint = 'https://wifire-data.sdsc.edu:9000'
access_key = os.getenv("ACCESS_KEY")
secret_key = os.getenv("SECRET_KEY")

fs = s3fs.S3FileSystem(key=access_key,
    secret=secret_key,
    client_kwargs={
        'endpoint_url': endpoint,
        'verify': False
    },
    skip_instance_cache=False
)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

name = 'quicfire.zarr'
bucket = 'burnpro3d/d'

# root = list(fs.ls(bucket))

simulation_paths = []
print(secret_key)

None


In [ ]:
def get_paths(rangestart,rangeend):
    global fs, bucket
    paths = fs.ls(bucket)
    for path in paths[rangestart:rangeend]:
        paths = fs.ls(path)
        for path in paths:
            get_child_directories(path)

def get_child_directories(path):
    global fs
    global simulation_paths
    paths = fs.ls(path)
    for p in paths:
        if "run_" in p:
            simulation_paths.append(p)

In [ ]:
print(len(root))
print("getting paths 0-259")
get_paths(0,7)

pprint(simulation_paths)
print(len(simulation_paths))

In [246]:
with open("paths.txt", "a") as file:
    for i in simulation_paths:
        file.write(i)
        file.write("\n")

In [247]:
len(simulation_paths)

638

In [248]:
def read_paths():
    paths = []
    with open("paths.txt","r") as f:
        paths = f.read().splitlines() 
    return paths

In [249]:
simulation_paths = read_paths()
len(simulation_paths)

26997

# Now Extract Data

In [250]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

bucket = "devin"
org = "SDSC internship"
# Store the URL of your InfluxDB instance
url= "http://localhost:8086"

client = influxdb_client.InfluxDBClient(
    url=url,
    token="FIX",
    org=org
)

# Write script
write_api = client.write_api(write_options=SYNCHRONOUS)

p = influxdb_client.Point("ravi").tag("location", "Czech").field("temperature", 89.9).field("height",6).field("age",8)
write_api.write(bucket=bucket, org=org, record=p)

In [251]:
json_payload = []
data = {
    "measurement": "simulation_run",
    "tags": {
        "wifire"
    },
    "time": "2022-10-22T12:52:03.903821-07:00",
    "fields": {}
}
# json_payload.append(data)
# client.write_points(json_payload)

In [255]:
KEEP_ATTRIBUTES = {
    'path': lambda d: None,
    'canopy_moisture': lambda d: d['canopy_moisture'],
    # 'dz',
    'extent': lambda d: d['extent'],
    'extent_fmt': lambda d: d['extent_fmt'],
    'fire_grid': lambda d: d['fire_grid'],
    'fuel': lambda d: d['fuel'],
    'ignition': lambda d: d['ignition'],
    'output': lambda d: d['output'],
    'resolution': lambda d: d['resolution'],
    'resolution_units': lambda d: d['resolution_units'],
    'run_binary': lambda d: d['run_binary'],
    'run_end': lambda d: d['run_end'],
    'run_max_mem_rss_bytes': lambda d: d['run_max_mem_rss_bytes'],
    'run_start': lambda d: d['run_start'],
    'seed': lambda d: d['seed'],
    'sim_time': lambda d: d['sim_time'],
    'surface_moisture': lambda d: d['surface_moisture'],
    'threads': lambda d: d['threads'],
    'timestep': lambda d: d['timestep'],
    'topo': lambda d: d['topo'],
    'wind_direction': lambda d: d['wind_direction'],
    'wind_speed': lambda d: d['wind_speed']
}
filenotfound = []

def get_df(count):
    global simulation_paths, KEEP_ATTRIBUTES, incomplete, filenotfound

    df = pd.DataFrame([], columns=KEEP_ATTRIBUTES.keys())

    i = 0
    for p in tqdm(simulation_paths[:count]):
        print(f"running loop path is '{p + '/' + name + '/.zattrs'}'")
    
        with fs.open(p + '/' + name + '/.zattrs') as f:
            data=json.load(f)
            
        print(data)
        row = []
        for k,expr in KEEP_ATTRIBUTES.items():
            try:
                value = KEEP_ATTRIBUTES[k](data)
            except KeyError:
                value = None
            row.append(value)
        row[0] = p
        print("adding row \t\n" ,row)
        df.loc[i] = row
        i+=1

            # filenotfound.append(p)
            # print("FileNotFound error on path {",p,"}")
    return df

pd.set_option('display.max_columns', None)
print(filenotfound)

[]


In [266]:
simulation_paths = read_paths()
df = get_df(3)

  0%|                                                                                    | 0/3 [00:00<?, ?it/s]

running loop path is 'burnpro3d/d/00/08/run_0008c20d-b78b-4a33-bd01-9635bdb61ec5/quicfire.zarr/.zattrs'


NoCredentialsError: Unable to locate credentials

In [239]:
df

,path,canopy_moisture,extent,extent_fmt,fire_grid,fuel,ignition,output,resolution,resolution_units,run_binary,run_end,run_max_mem_rss_bytes,run_start,seed,sim_time,surface_moisture,threads,timestep,topo,wind_direction,wind_speed


# Now Query for the Data we entered

In [104]:
query_api = client.query_api()
query = 'from(bucket:"devin")\
|> range(start: -10m)\
|> filter(fn:(r) => r._measurement == "simulation_run")'
result = query_api.query(org=org, query=query)

In [110]:
client.default_tags

In [98]:
results = []
for table in result:
  for record in table.records:
    results.append((record.get_field(), record.get_value()))

print(results)
# [(temperature, 25.3)]

[('age', 67), ('age', 90), ('age', 8), ('height', 42), ('height', 231), ('height', 231), ('height', 231), ('height', 78), ('height', 6), ('temperature', 29.3), ('temperature', 29.3), ('temperature', 29.3), ('temperature', 29.3), ('temperature', 8.9), ('temperature', 89.9)]
